Okay I'm going to be real with you
Wednesday was tough, I spent hours and hours and hours working on this project (and the other one) to no avail.
I salute past Ace for their efforts but I'm frustrated with their stupidity.
Yesterday I didn't get much done, I was putting out fires and making money
Today I had a full mental break down over dishes
But I'm ok now. I'm ready to give this another go.

Part 3 didn't succeed in much, but it did at least streamline the main code. We can go from there.

This notebook was written based on these resources:
https://www.kaggle.com/code/someadityamandal/bitcoin-time-series-forecasting/notebook
https://github.com/dataquestio/project-walkthroughs/tree/master/bitcoin_price
https://www.geeksforgeeks.org/time-series-forecasting-using-tensorflow/

Special thank you to David for, as always, having the best resources.

Imports for the old data processing first, then we'll bring in the stuff for up-to-date data processing later, after that we'll play with graphs.
(Counter intuitive I know. Graphs are definitely an important part of the data stuff, but I need to at least make sure my code runs first.)

In [3]:
import numpy as np
import pandas as pd
import keras.layers as layers
import keras.models as models

import datetime, pytz
from sklearn.preprocessing import MinMaxScaler

import os
print(os.listdir('./data'))

['coinbaseUSD.csv', 'bitstampUSD.csv']


Idk how exactly we want to do it but let's pre-train it on the old data? Ah, no ok so I suppose we'd want to train it all at once later.
Ok so let's type up the rest of the program and then we'll try it with a different kernel for different data processing.

In [ ]:
def dateparse(time_in_secs):
    # This one will have to change as well.
    # The new data isn't in seconds but rather hours?
    # IIRC
    # Also, new and improved dateparse function
    # I'll admit I got AI's help with this one
    return pytz.utc.localize(datetime.datetime.fromtimestamp(float(time_in_secs)))

data = pd.read_csv('./data/coinbaseUSD.csv', parse_dates=[0], date_parser=dateparse)

data.info()
print(data.head)

/tmp/ipykernel_205336/3754294470.py:7: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  data = pd.read_csv('./data/coinbaseUSD.csv', parse_dates=[0], date_parser=dateparse)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2099760 entries, 0 to 2099759
Data columns (total 8 columns):
 #   Column             Dtype              
---  ------             -----              
 0   Timestamp          datetime64[ns, UTC]
 1   Open               float64            
 2   High               float64            
 3   Low                float64            
 4   Close              float64            
 5   Volume_(BTC)       float64            
 6   Volume_(Currency)  float64            
 7   Weighted_Price     float64            
dtypes: datetime64[ns, UTC](1), float64(7)
memory usage: 128.2 MB
<bound method NDFrame.head of                         Timestamp     Open     High      Low    Close  \
0       2014-11-30 23:33:00+00:00   300.00   300.00   300.00   300.00   
1       2014-11-30 23:34:00+00:00      NaN      NaN      NaN      NaN   
2       2014-11-30 23:35:00+00:00      NaN      NaN      NaN      NaN   
3       2014-11-30 23:36:00+00:00      NaN      NaN      NaN

It may be worth it to pretrain our model? Not actually sure off the top of my head how that works.

In [7]:
# processing. I have to see how this does with the different data set later.
# Also, David's model was overfitting, 
# and I have to wonder if having such a granular dataset may be to blame.
# A worth while test at least. That'll be fun.

data['Volume_(BTC)'].fillna(value=0, inplace=True)
data['Volume_(Currency)'].fillna(value=0, inplace=True)
data['Weighted_Price'].fillna(value=0, inplace=True)

data['Open'].fillna(method='ffill', inplace=True)
data['High'].fillna(method='ffill', inplace=True)
data['Low'].fillna(method='ffill', inplace=True)
data['Close'].fillna(method='ffill', inplace=True)
# Also, exactly which dates are the ones with zeros?
# I feel there's a better way to process that data but IDK what it would be
# Well, ok vaguely. But idk off the top of my head how to do it.

print(data.head())

/tmp/ipykernel_205336/3391126903.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Volume_(BTC)'].fillna(value=0, inplace=True)
/tmp/ipykernel_205336/3391126903.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

                  Timestamp   Open   High    Low  Close  Volume_(BTC)  \
0 2014-11-30 23:33:00+00:00  300.0  300.0  300.0  300.0          0.01   
1 2014-11-30 23:34:00+00:00  300.0  300.0  300.0  300.0          0.00   
2 2014-11-30 23:35:00+00:00  300.0  300.0  300.0  300.0          0.00   
3 2014-11-30 23:36:00+00:00  300.0  300.0  300.0  300.0          0.00   
4 2014-11-30 23:37:00+00:00  300.0  300.0  300.0  300.0          0.00   

   Volume_(Currency)  Weighted_Price  
0                3.0           300.0  
1                0.0             0.0  
2                0.0             0.0  
3                0.0             0.0  
4                0.0             0.0  


/tmp/ipykernel_205336/3391126903.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Open'].fillna(method='ffill', inplace=True)
/tmp/ipykernel_205336/3391126903.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['Open'].fillna(method='ffill', inplace=True)
/tmp/ipykernel_205336/3391126903.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
T

Oof those sure are a lot of warnings. I'll sort those out later.
There should be a way to disable the warnings.

In [8]:
start = datetime.datetime(2015, 1, 1, 0,0,0,0, pytz.UTC)
end = datetime.datetime(2018, 11,11,0,0,0,0, pytz.UTC)
# This is also something that we'll need to change
# Though I suppose it'll just be changing the end date.

weekly_rows = data[(data['Timestamp'] >= start) & (
    data['Timestamp'] <= end)].groupby(
        [pd.Grouper(key='Timestamp', freq='W-MON')]
    ).first().reset_index()

print(weekly_rows.head())
# Ok that line is GOING to need to be more understandable 
# in order for me to mess with it
# That being said, I think it's just the data object I need to look at
# And pd.Grouper.

                  Timestamp    Open    High     Low   Close  Volume_(BTC)  \
0 2015-01-12 00:00:00+00:00  360.00  360.00  360.00  360.00         0.010   
1 2015-01-19 00:00:00+00:00  260.00  260.00  260.00  260.00         0.000   
2 2015-01-26 00:00:00+00:00  225.51  225.51  225.51  225.51         0.000   
3 2015-02-02 00:00:00+00:00  259.99  259.99  259.99  259.99         0.300   
4 2015-02-09 00:00:00+00:00  239.28  239.50  239.28  239.50        20.675   

   Volume_(Currency)  Weighted_Price  
0           3.600000      360.000000  
1           0.000000        0.000000  
2           0.000000        0.000000  
3          77.997000      259.990000  
4        4951.178693      239.476599  


Those timestamps frustrate me.

In [9]:
data = pd.read_csv('./data/coinbaseUSD.csv', parse_dates=[0], date_parser=dateparse)
data['Timestamp'] = data['Timestamp'].dt.tz_localize(None)

data = data.groupby([pd.Grouper(key='Timestamp', freq='H')]).first().reset_index()
# Ok so we DO go by Hour at a later point.
# Maybe we should try the day
# It's 4 years of data after all
# That's 35040 points of data. Just for this bit.
# David's was 78840
# It COULD however be 3285 points of data
# If done by the day
# IDK future Ace. I leave that in your hands.

data = data.set_index('Timestamp')
data = data[['Weighted_Price']]
data['Weighted_Price'].fillna(method='ffill', inplace=True)

/tmp/ipykernel_205336/1170822903.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  data = pd.read_csv('./data/coinbaseUSD.csv', parse_dates=[0], date_parser=dateparse)
/tmp/ipykernel_205336/1170822903.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  data = data.groupby([pd.Grouper(key='Timestamp', freq='H')]).first().reset_index()
/tmp/ipykernel_205336/1170822903.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or d

In [10]:
split_date = '25-Jun-2018'
# He leaves the last 6 months to test with.
# Idk what the requirements are for the prediction model
# But surely we run less than 6 months, right?
# IDK maybe people plan their investments 6 months ahead
# Not how I'd roll my dice but you do you.

data_train = data.loc[data.index <= split_date].copy()
data_test = data.loc[data.index > split_date].copy()

In [11]:
# Oh look! More preprocessing. Have fun future Ace.
training_set = data_train.values
training_set = np.reshape(training_set, (len(training_set), 1))

sc = MinMaxScaler()
training_set = sc.fit_transform(training_set)
X_train = training_set[0:len(training_set)-1]
y_train = training_set[1:len(training_set)]
X_train = np.reshape(X_train, (len(X_train), 1, 1))

In [13]:
# Time to build the model!!
# Honestly I think we'll scrap the model as we have it 
# and find a different example.
# ooorrrr maybe a few. I'm just curios.
# Oh great
# All the tutorials have different preprocessing
# Who would've guessed.
# That's fine I'll make it work

# model_1 = models.Sequential([
#   LSTM(50, activation='relu', input_shape=(seq_length, 1))
# ])

model1 = models.Sequential()

model1.add(layers.LSTM(128, activation='relu', input_shape=(1, 1)))
model1.add(layers.Dense(1))
model1.compile(loss='mean_squared_error', optimizer='adam')
model1.fit(X_train, y_train, epochs=100, batch_size=50, verbose=2)

model2 = models.Sequential()

# This one's more promising than the other's I've seen
model2.add(layers.LSTM(100, return_sequences=True))
model2.add(layers.LSTM(100, return_sequences=True))
model2.add(layers.LSTM(50))
model2.add(layers.Dense(8, activation='relu'))
model2.add(layers.Dense(1, activation= 'linear'))
# Interesting that this one does two different activation functions
# Suppose that makes sense though

# Ok last one is that weird amalgamation I did last time.
# Copy and pasting isn't cheating when it's your own code
model3 = models.Sequential()

model3.add(layers.LSTM(128, activation='relu', input_shape=(1, 1)))
model3.add(layers.Dropout(0.2))
model3.add(layers.Dense(1))
model3.add(layers.Dropout(0.2))
model3.add(layers.Dense(1))
model3.compile(loss='mean_squared_error', optimizer='adam')
model3.fit(X_train, y_train, epochs=100, batch_size=50, verbose=2)

model1.compile(loss='mean_squared_error', optimizer='adam')
model2.compile(loss='mean_squared_error', optimizer='adam')
model3.compile(loss='mean_squared_error', optimizer='adam')

model1.fit(X_train, y_train, epochs=100, batch_size=50, verbose=2)
model2.fit(X_train, y_train, epochs=100, batch_size=50, verbose=2)
model3.fit(X_train, y_train, epochs=100, batch_size=50, verbose=2)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
625/625 - 3s - 6ms/step - loss: 0.0067
Epoch 2/100
625/625 - 2s - 2ms/step - loss: 5.7712e-05
Epoch 3/100
625/625 - 2s - 3ms/step - loss: 2.9065e-05
Epoch 4/100
625/625 - 2s - 3ms/step - loss: 1.5137e-05
Epoch 5/100
625/625 - 2s - 3ms/step - loss: 1.1651e-05
Epoch 6/100
625/625 - 3s - 4ms/step - loss: 1.1670e-05
Epoch 7/100
625/625 - 2s - 3ms/step - loss: 1.1545e-05
Epoch 8/100
625/625 - 2s - 3ms/step - loss: 1.1883e-05
Epoch 9/100
625/625 - 2s - 3ms/step - loss: 1.2015e-05
Epoch 10/100
625/625 - 2s - 3ms/step - loss: 1.1975e-05
Epoch 11/100
625/625 - 2s - 3ms/step - loss: 1.2638e-05
Epoch 12/100
625/625 - 2s - 3ms/step - loss: 1.2498e-05
Epoch 13/100
625/625 - 2s - 4ms/step - loss: 1.2313e-05
Epoch 14/100
625/625 - 3s - 4ms/step - loss: 1.2597e-05
Epoch 15/100
625/625 - 2s - 3ms/step - loss: 1.2325e-05
Epoch 16/100
625/625 - 2s - 3ms/step - loss: 1.2342e-05
Epoch 17/100
625/625 - 2s - 3ms/step - loss: 1.2364e-05
Epoch 18/100
625/625 - 2s - 3ms/step - loss: 1.2400e-05
Epoch

In [14]:
model1.summary()
model2.summary()
model3.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 128)            │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 200,069 (781.52 KB)

 Trainable params: 66,689 (260.50 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 128)            │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 200,075 (781.55 KB)

 Trainable params: 66,691 (260.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 133,384 (521.04 KB)

Mmkay while those are training let's start working on the actual recent data.
This is where David's resource comes in a clutch.

In [31]:
# From what I can tell it's just getting the CSV for the data
# We should be able to do the same preprocessing as before, but with those edits I made
# I should have waited to train the models, but we'll do that in V5

import yfinance as yf

btc_ticker = yf.Ticker("BTC-USD")

if os.path.exists("btc.csv"):
    btc = pd.read_csv("btc.csv", index_col=0)
else:
    btc = btc_ticker.history(period="max")
    btc.to_csv("btc.csv")

btc

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2014-09-17 00:00:00+00:00,465.864014,468.174011,452.421997,457.334015,21056800,0.0,0.0
2014-09-18 00:00:00+00:00,456.859985,456.859985,413.104004,424.440002,34483200,0.0,0.0
2014-09-19 00:00:00+00:00,424.102997,427.834991,384.532013,394.795990,37919700,0.0,0.0
2014-09-20 00:00:00+00:00,394.673004,423.295990,389.882996,408.903992,36863600,0.0,0.0
2014-09-21 00:00:00+00:00,408.084991,412.425995,393.181000,398.821014,26580100,0.0,0.0
...,...,...,...,...,...,...,...
2024-11-11 00:00:00+00:00,80471.414062,89604.500000,80283.250000,88701.484375,117966845037,0.0,0.0
2024-11-12 00:00:00+00:00,88705.562500,89956.882812,85155.109375,87955.812500,133673285375,0.0,0.0
2024-11-13 00:00:00+00:00,87929.968750,93434.351562,86256.929688,90584.164062,123559027869,0.0,0.0


We're now at our models finshing up being trained
This is going to take almost 3 times as long with the recent data.
New mission, get the models training before long break
While doing that, we work on vizualizing.
Then after long break we write the paper.
We should have enough notes to write at least part of it quickly.
For now let's build the new preprocessing.

In [33]:
# Mmkay so the dateparse doesn't determine the seconds. 
# Also I think the data pulls by the day?
# Also also, it looks like there's an easier way to preprocess. 
# I just have to work that in to feeding my models.
# It also only pulls dividends and stock splits
# So maybe.
# Wait this isn't tensorflow set up. He's working with pytorch IIRC
# Mmkay we're doing it the original way
#   NOPE
#   not-so-distant-future-Ace here to tell you we are NOT doing the old one
#   I'm tired lol

# btc.index = pd.to_datetime(btc.index)

# del btc['Dividends']
# del btc['Stock Splits']

# btc.columns = [c.islower() for c in btc.columns]

# We're not doing it that way either apparantly
if isinstance(btc.index, pd.DatetimeIndex):
    btc.index = pd.to_datetime(btc.index)
else:
    btc['date'] = pd.to_datetime(btc.index)

btc.set_index(['date'], inplace=True)

del btc['Dividends']
del btc['Stock Splits']

btc.columns = [c.lower() for c in btc.columns]

In [34]:
# # As we have our models already built for all the fields, we'll keep using them
# # data['volume_(BTC)'].fillna(value=0, inplace=True)
# btc['Volume_(Currency)'].fillna(value=0, inplace=True)
# btc['Weighted_Price'].fillna(value=0, inplace=True)

# btc['Open'].fillna(method='ffill', inplace=True)
# btc['High'].fillna(method='ffill', inplace=True)
# btc['Low'].fillna(method='ffill', inplace=True)
# btc['Close'].fillna(method='ffill', inplace=True)

# # All of those may be redundant
# # so we'll try not running this first

# print(btc.head())

# for col in ['Volume_(Currency)', 'Weighted_Price']:
#     btc[col].fillna(value=0, inplace=True)

price_columns = ['Open', 'High', 'Low', 'Close']
for col in price_columns:
    btc[col].fillna(method='ffill', inplace=True)

print(btc.head())

KeyError: 'Open'

I was worried that the data was built with completely different fields.
Turns out it's just that first one.
Vibes

In [27]:
# I have still not fixed the warnings
# Love you future Ace <3

start = datetime.datetime(2015, 1, 1, 0,0,0,0, pytz.UTC)
end = datetime.datetime(2024, 5, 5, 0,0,0,0, pytz.UTC)
# Honestly Idk if it's month then year or year then month
# Regardless the 5th of may isn't a bad place to split

weekly_rows = btc[(btc.index >= start) & (
    btc.index <= end)].groupby(
        [pd.Grouper(key=btc.index, freq='W-MON')]
    ).first().reset_index()
# WAIT DOES THAT FREQUENCY MEAN WEEK TO MONTH???
# GODDAMIT ACE YOU'RE SMARTER THAN THIS
# Ok so it's definitely not the granular data that's the problem
# Probably
# That being said, I do have to wonder how months would work out.
# Anyway, time to make models 1b, 2b, and 3b

print(weekly_rows.head())

TypeError: unhashable type: 'DatetimeIndex'

In [ ]:
# Ok so the data did not save where I expected it to
# That's fine.

data = pd.read_csv('btc.csv', parse_dates=[0], date_parser=dateparse)
data['Timestamp'] = data['Timestamp'].dt.tz_localize(None)

data = data.groupby([pd.Grouper(key='Timestamp', freq='W-MON')]).first().reset_index()

data = data.set_index('Timestamp')
data = data[['Weighted_Price']]
data['Weighted_Price'].fillna(method='ffill', inplace=True)

/tmp/ipykernel_205336/1007552186.py:4: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  data = pd.read_csv('btc.csv', parse_dates=[0], date_parser=dateparse)


ValueError: could not convert string to float: '2014-09-17 00:00:00+00:00'

You know what? We're moving to V6 with the new data processing in mind.